In [25]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import lightgbm as lgb

In [118]:
def generate_samples(n, cut, N, d):
    data = []  
    for k in range(0, N):
        p = np.random.rand()
        if p<=cut:
            spectre = np.diag((np.random.rand((n)) - 1)*d/2)
            label = 1
        else:
            spectre = np.diag((np.random.rand((n)) - 1/2)*d)
            label = 0   
        new_basis = d*np.random.rand(n, n)
        system = np.linalg.inv(new_basis).dot(spectre.dot(new_basis))
        temp_list = list(system.reshape(1, -1).flatten())
        temp_list.append(label)
        data.append(temp_list)

    data = np.matrix(data)
    X = np.asarray(data[:, :-1])
    Y = np.asarray(data[:, -1]).flatten().astype('int')
    return X, Y

In [123]:
%%time
n = 2 #dimension of systems
cut = 1/2 #procent of stable samples
N = 500000 #number of samples
d = 100 #~dispersion
X_train, Y_train = generate_samples(n, cut, N, d)
X_test, Y_test = generate_samples(n, cut, N//5, d)

Wall time: 38.6 s


In [121]:
%%time
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

Wall time: 201 ms


In [122]:
Y_pred = clf.predict(X_test)
acc_score = accuracy_score(Y_test, Y_pred)
roc_score = roc_auc_score(Y_test, Y_pred)
cm = confusion_matrix(Y_test, Y_pred)
print('ACC:', acc_score)
print('ROC_AUC:', roc_score)
print('\nCONFUSION MATRIX\n', cm / cm.sum(axis=1))

ACC: 0.768
ROC_AUC: 0.7703213762693316

CONFUSION MATRIX
 [[0.73977695 0.3030303 ]
 [0.17100372 0.8008658 ]]


In [128]:
%%time
train_data = lgb.Dataset(X_train, label = Y_train)
test_data = lgb.Dataset(X_test, label = Y_test)
param = {
        'num_leaves':50, 
        'max_depth':100,
        'num_trees':100, 
        'num_iterations':1000,
        'learning_rate' : 0.1,
        'objective':'binary',
        'metric':['auc', 'acc'],
        'verbose':1
        }

num_round = 50
bst = lgb.train(param, train_data, num_round)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:111: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Wall time: 20.9 s


In [127]:
Y_pred = (bst.predict(X_test)>0.5).astype('int')
acc_score = accuracy_score(Y_test, Y_pred)
roc_score = roc_auc_score(Y_test, Y_pred)
cm = confusion_matrix(Y_test, Y_pred)
print('ACC:', acc_score)
print('ROC_AUC:', roc_score)
print('\nCONFUSION MATRIX\n', cm / cm.sum(axis=1))

ACC: 0.83071
ROC_AUC: 0.8303608026393882

CONFUSION MATRIX
 [[0.73598314 0.26207034]
 [0.07582054 0.92473847]]


In [96]:
(Y_pred>0.5).astype('int')

array([0, 0, 1, ..., 1, 0, 1])

In [130]:
lst = [2]

In [134]:
system

[]

[]